In [1]:
import pandas as pd
import numpy as np



In [2]:
amazon = pd.read_csv("AMAZON_daily.csv")
amazon['Date'] = pd.to_datetime(amazon['Date'])
amazon.head()



,Date,Open,High,Low,Close,Adj Close,Volume
0,1997-05-15,0.121875,0.125000,0.096354,0.097917,1443120000,0.097917
1,1997-05-16,0.098438,0.098958,0.085417,0.086458,294000000,0.086458
2,1997-05-19,0.088021,0.088542,0.081250,0.085417,122136000,0.085417
3,1997-05-20,0.086458,0.087500,0.081771,0.081771,109344000,0.081771
4,1997-05-21,0.081771,0.082292,0.068750,0.071354,377064000,0.071354


In [3]:
amazon.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
7007,2025-03-24,200.000000,203.639999,199.949997,203.259995,41625400,203.259995
7008,2025-03-25,203.600006,206.210007,203.220001,205.710007,31171200,205.710007
7009,2025-03-26,205.839996,206.009995,199.929993,201.130005,32855300,201.130005
7010,2025-03-27,200.889999,203.789993,199.279999,201.360001,27317700,201.360001
7011,2025-03-28,198.419998,199.259995,191.880005,192.720001,52462100,192.720001


In [4]:
target = 'Close'
train_size = int(len(amazon) * 0.8)
train = amazon.iloc[:train_size].copy()
test = amazon.iloc[train_size:].copy()

In [8]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7012 entries, 0 to 7011
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       7012 non-null   datetime64[ns]
 1   Open       7012 non-null   float64       
 2   High       7012 non-null   float64       
 3   Low        7012 non-null   float64       
 4   Close      7012 non-null   float64       
 5   Adj Close  7012 non-null   int64         
 6   Volume     7012 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 383.6 KB


#### ***Model 1: Naive Forecast***

In [5]:
y_true = test[target].values
last_train = train[target].iloc[-1]

y_pred_naive = np.repeat(last_train, len(test))

#### ***Model 2: LSTM***

In [6]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# -------------------------
# 1. SCALE TRAIN ONLY
# -------------------------
train_vals = train[[target]].values
test_vals  = test[[target]].values

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_vals)
test_scaled  = scaler.transform(test_vals)

# -------------------------
# 2. CREATE SEQUENCES
# -------------------------
def create_sequences(values, window=30):
    X, y = [], []
    for i in range(window, len(values)):
        X.append(values[i-window:i])
        y.append(values[i])
    return np.array(X), np.array(y)

window = 30

# Train sequences
X_train, y_train = create_sequences(train_scaled, window)

# Test sequences: last window of train + entire test
test_input = np.concatenate([train_scaled[-window:], test_scaled])
X_all_test, y_all_test = create_sequences(test_input, window)

# Keep only sequences whose target is in the test set
X_test = X_all_test[-len(test):]
y_test_lstm = y_all_test[-len(test):]

# Reshape
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test  = X_test.reshape((X_test.shape[0],  X_test.shape[1],  1))


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

lstm = Sequential([
    LSTM(64, return_sequences=True, input_shape=(window,1)),
    LSTM(32),
    Dense(1)
])

lstm.compile(optimizer='adam', loss='mse')

# Early stopping
es = EarlyStopping(patience=5, restore_best_weights=True)

history = lstm.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[es],
    verbose=1
)

# Predictions
y_pred_lstm_scaled = lstm.predict(X_test)
y_pred_lstm = scaler.inverse_transform(y_pred_lstm_scaled)


2025-11-21 11:55:57.871635: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-11-21 11:55:57.871749: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-21 11:55:57.871754: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-21 11:55:57.871911: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-21 11:55:57.871921: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefe

Epoch 1/50


2025-11-21 11:55:58.479863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-11-21 11:55:58.484423: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 5.3076e-04 - val_loss: 0.0018
Epoch 2/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 3.4789e-05 - val_loss: 0.0011
Epoch 3/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 3.2680e-05 - val_loss: 0.0025
Epoch 4/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 3.2479e-05 - val_loss: 9.5173e-04
Epoch 5/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 2.7288e-05 - val_loss: 0.0012
Epoch 6/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 2.6328e-05 - val_loss: 7.2736e-04
Epoch 7/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 2.5717e-05 - val_loss: 6.7655e-04
Epoch 8/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 2.4370e-05 - val_loss: 8.7072e-04
Epoch 9/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 2.4829e-05 - val_loss: 6.3395e-04
Epoch 10/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 2.0134e-05 - val_loss: 0.0011
Epoch 11/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 2.4577e-05 - val_l

2025-11-21 11:57:30.198987: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


#### ***Model 3: NHITS***

In [20]:
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS

nf_train = train.rename(columns = {'Date':'ds', target: 'y'}).copy()
nf_train['unique_id'] = 'AMZN'


nf_test = test.rename(columns = {'Date':'ds', target: 'y'}).copy()
nf_test['unique_id'] = 'AMZN'


nf_train = nf_train[['unique_id', 'ds', 'y']]
nf_test = nf_test[['unique_id', 'ds', 'y']]


In [23]:
nfmodel = NHITS(input_size = 30, h = 1, max_steps = 20)

nf = NeuralForecast(models = [nfmodel], freq = 'D')
nf.fit(nf_train)

forecast = nf.predict(nf_test).reset_index()
y_pred_nhits = forecast['NHITS'].values[:len(test)]

Seed set to 1
/opt/anaconda3/lib/python3.12/site-packages/neuralforecast/common/_base_model.py:591: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.775     Total estimated model params size (MB)
34        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=20` reached.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

#### ***Model 4: XGBoost***

In [15]:
def make_lags(df, target, lags=10):
    df = df.copy()
    for i in range(1, lags+1):
        df[f'lag_{i}'] = df[target].shift(i)
    return df.dropna()


In [16]:
amazon_feat = amazon.copy()
amazon_feat['return'] = amazon_feat[target].pct_change()
amazon_feat['volatility'] = amazon_feat['return'].rolling(5).std()
amazon_feat['weekday'] = amazon_feat['Date'].dt.weekday
amazon_feat = amazon_feat.dropna()


In [17]:
lag_df = make_lags(amazon_feat, target, lags=10)
lag_train = lag_df.iloc[:train_size]
lag_test  = lag_df.iloc[train_size:]


In [18]:
from xgboost import XGBRegressor

X_train = lag_train.drop(columns=[target, "Date"])
y_train = lag_train[target]

X_test = lag_test.drop(columns=[target, "Date"])
y_test_xgb = lag_test[target]

xgb = XGBRegressor(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror'
)

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)


In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def metrics(y, yhat):
    y = np.asarray(y).reshape(-1)
    yhat = np.asarray(yhat).reshape(-1)
    assert len(y) == len(yhat), f"length mismatch: y={len(y)}, yhat={len(yhat)}"
    mae = mean_absolute_error(y, yhat)
    mse = mean_squared_error(y, yhat)
    rmse = mse**0.5
    return mae, mse, rmse

# Make sure N-HiTS predictions are 1D
y_pred_nhits = np.asarray(y_pred_nhits).reshape(-1)

# -------------------------------
# 1. Naive  (uses original test)
# -------------------------------
y_true_naive = test["Close"].values
mae_naive, mse_naive, rmse_naive = metrics(y_true_naive, y_pred_naive)

# -------------------------------
# 2. LSTM  (uses y_test_lstm)
# -------------------------------
y_true_lstm = y_test_lstm.reshape(-1)
y_pred_lstm_1d = y_pred_lstm.reshape(-1)
mae_lstm, mse_lstm, rmse_lstm = metrics(y_true_lstm, y_pred_lstm_1d)

# -------------------------------
# 3. N-HiTS  (must use nf_test['y'])
# -------------------------------
y_true_nhits = nf_test["y"].values   # correct ground truth for NeuralForecast
y_true_nhits = y_true_nhits[:len(y_pred_nhits)]  # align lengths

mae_nhits, mse_nhits, rmse_nhits = metrics(y_true_nhits, y_pred_nhits)

# -------------------------------
# 4. XGBoost  (uses y_test_xgb)
# -------------------------------
y_true_xgb = y_test_xgb.values
y_pred_xgb_1d = np.asarray(y_pred_xgb).reshape(-1)
mae_xgb, mse_xgb, rmse_xgb = metrics(y_true_xgb, y_pred_xgb_1d)

# -------------------------------
# Results Table
# -------------------------------
results = pd.DataFrame({
    "Model": ["Naive", "LSTM", "N-HiTS", "XGBoost"],
    "MAE":  [mae_naive, mae_lstm, mae_nhits, mae_xgb],
    "MSE":  [mse_naive, mse_lstm, mse_nhits, mse_xgb],
    "RMSE": [rmse_naive, rmse_lstm, rmse_nhits, rmse_xgb]
})

results


,Model,MAE,MSE,RMSE
0,Naive,58.067178,4698.870464,68.548308
1,LSTM,138.562243,20199.425876,142.124684
2,N-HiTS,103.160576,10642.104413,103.160576
3,XGBoost,48.105997,3502.498386,59.181909
